In [464]:
import requests
import pandas as pd
import numpy as np
import json

In [449]:
OAuth = "Bearer BQDYnfv3kS29rnbhu5LJ0AAm1z-bgSi99gXDCMtLrwzeiT8olSzpGrGTfIvdUnBqFTAvpsF806A-6hv3DzoaDS-eL-50tE_TtKbHYC53Fy948VRwO9tjTNbWtZHEqMZHxbKS5d49zIJQjDpqim2H46oPYp2EwHxoMy6mg-B3P9Nc1nLpVqs"
headers = {"Accept" : "application/json",
                        "Content-Type" : "application/json",
                        "Authorization": OAuth}

In [450]:
def get_playlists(playlist_type, country_code, limit):
    
    '''
    Returns a dictionary of containing the given amount of playlists of the provided type from the 
    country matching the country code provided.
    '''
    
    playlists = (requests.get(url=f"https://api.spotify.com/v1/browse/categories/{playlist_type}/playlists?country={country_code}&limit={limit}",
                              headers = headers)).json()
    
    return playlists

In [451]:
def extract_playlist_ids(playlists):
    
    ''' Extract the ids of the playlists to be able to query the tracks contained in the playlist using Spotify API. '''
    
    ids = []
    playlist_info = playlists['playlists']['items']
    for i in range(len(playlist_info)):
        ids.append(playlist_info[i]['id'])
    
    return ids

In [452]:
def extract_track_info(playlist_ids):
    
    ''' Extract all of the relevant information about the tracks in a playlist, given a list a playlist ids. '''
    
    track_info = []

    for playlist in playlist_ids:
        tracks = (requests.get(url=f"https://api.spotify.com/v1/playlists/{playlist}/tracks",
                               headers = headers)).json()            
  
        fields = {"id", "name", "duration_ms", "popularity" }
        for i in range(len(tracks['items'])):
            if tracks['items'][i]['track']:
                info = { key:value for key,value in tracks['items'][i]['track'].items() if key in fields}
                track_info.append(info)
            else:
                print('BARK BARK BARK >:(')
    
    return track_info

In [453]:
def get_tracks(playlist_type, country_code, limit):
    
    '''
    Returns a list containing the information for various tracks.
    playlist_type: The type of playlists where you want to query songs
    country_code: The country code of the country where you want to find the playlists
    limit: The amount of playlists you want to extract tracks from
    '''
    
    playlists = get_playlists(playlist_type, country_code, limit)
    playlist_ids = extract_playlist_ids(playlists)
    track_info = extract_track_info(playlist_ids)
    
    return track_info

In [454]:
def get_track_features(track_ids):
    '''
    Given an array of track ids extract the audio features of the track.
    '''
    track_features =[]
    lower_bound = 0
    upper_bound = 60
    while upper_bound < len(track_ids):
        track_ids = ','.join(US_workout_tracks['id'][lower_bound:upper_bound])
        features = requests.get(url=f"https://api.spotify.com/v1/audio-features?ids={track_ids}",
                                params={'ids':track_ids},
                                headers=headers)
        lower_bound += 60
        upper_bound += 60
        track_features.extend(features.json()['audio_features'])
    return track_features

In [455]:
workout_tracks = pd.DataFrame(get_tracks('workout', 'US', 5))
workout_tracks = workout_tracks[['id', 'name', 'duration_ms', 'popularity']]

display(workout_tracks.head())
print(workout_tracks.shape)

,id,name,duration_ms,popularity
0,2KH16WveTQWT6KOG9Rg6e2,Eye of the Tiger,245640,75
1,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,208400,83
2,0pqnGHJpmpxLKifKRmU6WP,Believer,204346,87
3,5BIMPccDwShpXq784RJlJp,Enter Sandman,331573,73
4,57BrRMwf9LrcmuOsyGilwr,Crawling,208960,73


(370, 4)


In [456]:
workout_track_features = pd.DataFrame(get_track_features(US_workout_tracks['id']))

In [457]:
# Keeping only the important audio features
workout_track_features = workout_track_features[['id', 'acousticness', 'danceability','energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]
display(workout_track_features.head(5))
print(workout_track_features.shape)

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2KH16WveTQWT6KOG9Rg6e2,0.13200,0.817,0.599,0.000311,0,0.0873,-9.249,0,0.0328,108.873,4,0.548
1,2zYzyRzz6pRmhPzyfMEC8s,0.05910,0.573,0.913,0.001730,6,0.1560,-4.793,0,0.1320,115.715,4,0.422
2,0pqnGHJpmpxLKifKRmU6WP,0.06220,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
3,5BIMPccDwShpXq784RJlJp,0.00206,0.579,0.824,0.009030,6,0.0590,-8.710,0,0.0300,123.331,4,0.635
4,57BrRMwf9LrcmuOsyGilwr,0.04660,0.580,0.702,0.000003,4,0.5360,-5.565,1,0.0337,105.076,4,0.299


(370, 13)


In [458]:
# Merge workout tracks so that all info is consolidated 
workout_tracks = pd.merge(workout_tracks, workout_track_features, on="id", how="inner").set_index('id')
workout_tracks.head()

,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,,
2KH16WveTQWT6KOG9Rg6e2,Eye of the Tiger,245640,75,0.1320,0.817,0.599,0.000311,0,0.0873,-9.249,0,0.0328,108.873,4,0.548
2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,208400,83,0.0591,0.573,0.913,0.001730,6,0.1560,-4.793,0,0.1320,115.715,4,0.422
0pqnGHJpmpxLKifKRmU6WP,Believer,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
0pqnGHJpmpxLKifKRmU6WP,Believer,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
0pqnGHJpmpxLKifKRmU6WP,Believer,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666


In [463]:
print("Number of unique tracks:", workout_tracks.name.nunique())
print("Number of tracks:", len(workout_tracks))

Number of unique tracks: 338
Number of tracks: 396


### At this point we've extracted all of our necessary information. Note that there are repeated tracks, which makes sense since popular songs tend to be present in many different playlists.